In [1]:
import db_connection as db_conn
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, to_timestamp, isnan, count
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql import functions as F

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
db_config = db_conn.config_gsql
# Create a SparkSession
spark = SparkSession.builder \
    .appName("process_tx") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "32g") \
    .getOrCreate()

23/06/19 18:54:48 WARN Utils: Your hostname, NatRng-MBP.local resolves to a loopback address: 127.0.0.1; using 10.40.130.60 instead (on interface en0)
23/06/19 18:54:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/19 18:54:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
url = f"jdbc:mariadb://{db_config['host']}:{db_config['port']}/{db_config['database']}"
user = db_config['user']
password = db_config['password']
transactions_df = spark.read.format('jdbc').options(url=url, dbtable='Transactions', user=user, password=password).load()
block_df = spark.read.format('jdbc').options(url=url, dbtable='Blocks', user=user, password=password).load()
address_df = spark.read.format('jdbc').options(url=url, dbtable='Addresses', user=user, password=password).load()
category_df = spark.read.format('jdbc').options(url=url, dbtable='TxCategories', user=user, password=password).load()
contract_df = spark.read.format('jdbc').options(url=url, dbtable='Contracts', user=user, password=password).load()

Outgoing and incoming Transactions

In [4]:
outgoing_transactions = transactions_df.groupby("from_id")\
                             .agg(F.count("tx_id").alias("outgoing_tx_count"))

incoming_transactions = transactions_df.filter(col("to_id").isNotNull()).groupby("to_id")\
                             .agg(F.count("tx_id").alias("incoming_tx_count"))

outgoing_transactions = outgoing_transactions.withColumnRenamed("from_id", "address_id")
incoming_transactions = incoming_transactions.withColumnRenamed("to_id", "address_id")

transactions_count = outgoing_transactions.join(incoming_transactions, "address_id", "outer")
transactions_count = transactions_count.fillna(0)

Time diff between first and last transaction

In [5]:
outgoing_timestamps = address_df.join(transactions_df, address_df.address_id == transactions_df.from_id, "left").select("address_id", "timestamp")
incoming_timestamps = address_df.join(transactions_df, address_df.address_id == transactions_df.to_id, "left").select("address_id", "timestamp")

all_timestamps = outgoing_timestamps.union(incoming_timestamps)

# Group by address_id and calculate min, max timestamp
transactions_timestamps = all_timestamps.groupBy("address_id")\
                                          .agg(F.min("timestamp").alias("first_tx_timestamp"),
                                               F.max("timestamp").alias("last_tx_timestamp"))

# Calculate the time difference for each address_id in minutes
transactions_timestamps = transactions_timestamps.withColumn(
    "time_difference",
    F.col("last_tx_timestamp").cast("long") - F.col("first_tx_timestamp").cast("long"))

transactions_timestamps = transactions_timestamps.withColumn(
    "time_difference_in_minutes",
    (F.col("time_difference") / 60))

# Show the results
transactions_timestamps = transactions_timestamps.select("address_id", "time_difference_in_minutes")

Contract Creation

In [6]:
contracts_created  = contract_df.join(transactions_df, contract_df['tx_id'] == transactions_df['tx_id'], "left")\
                                .select(contract_df["contract_id"], contract_df["tx_id"], transactions_df["from_id"])

contracts_created = contracts_created.groupBy("from_id").agg(F.count("tx_id").alias("contracts_created"))
contracts_created = contracts_created.withColumnRenamed("from_id", "address_id")

Get Unique Received From Addresses

In [7]:
unique_received_transactions = transactions_df.filter(col("to_id").isNotNull()).groupBy("to_id").agg(F.countDistinct("from_id").alias("unique_received_transactions"))
unique_received_transactions = unique_received_transactions.withColumnRenamed("to_id", "address_id")

In [8]:
eth_transactions = transactions_df.filter(col("asset") == "ETH").filter(col("to_id").isNotNull()) \
                    .fillna({"asset_value": 0}) \
                    .groupBy("to_id").agg(F.sum("asset_value").alias("total_incoming_eth"))
eth_transactions = eth_transactions.withColumnRenamed("to_id", "address_id")

erc20 transfers

In [9]:
#cast erc_20_tnx asset_value to float
erc_20_tnx = transactions_df.filter(col("category_id") == 3)
erc_20_tnx = erc_20_tnx.withColumn("asset_value", erc_20_tnx["asset_value"].cast(FloatType()))
outgoing_erc_20_tnx = erc_20_tnx.groupBy("from_id").agg(F.count("tx_id").alias("outgoing_erc_20_tnx"))
incoming_erc_20_tnx = erc_20_tnx.filter(col("to_id").isNotNull()).groupBy("to_id").agg(F.count("tx_id").alias("incoming_erc_20_tnx"))
outgoing_erc_20_tnx = outgoing_erc_20_tnx.withColumnRenamed("from_id", "address_id")
incoming_erc_20_tnx = incoming_erc_20_tnx.withColumnRenamed("to_id", "address_id")
total_erc_20_tnx = outgoing_erc_20_tnx.join(incoming_erc_20_tnx, "address_id", "outer").fillna(0)
total_erc_20_tnx = total_erc_20_tnx.withColumn("total_erc_20_tnx", F.col("outgoing_erc_20_tnx") + F.col("incoming_erc_20_tnx"))
total_erc_20_tnx = total_erc_20_tnx.select("address_id", "total_erc_20_tnx")

ERC 20 ETH OUT

In [10]:
erc20eth_out = erc_20_tnx.filter(col("asset") == "ETH")\
                .fillna({"asset_value": 0}) \
                .groupBy("from_id").agg(F.sum("asset_value").alias("total_outgoing_erc20eth"))
erc20eth_out = erc20eth_out.withColumnRenamed("from_id", "address_id")

min_erc20token_in = erc_20_tnx.filter(col("to_id").isNotNull()).groupBy("to_id") \
                    .min("asset_value").withColumnRenamed("min(asset_value)", "min_erc20token_in")
min_erc20token_in = min_erc20token_in.withColumnRenamed("to_id", "address_id")


num_unique_erc20tokens_out = erc_20_tnx.groupBy("from_id") \
                            .agg(F.countDistinct("asset").alias("num_unique_erc20tokens_out"))
num_unique_erc20tokens_out = num_unique_erc20tokens_out.withColumnRenamed("from_id", "address_id")

Merge Data

In [11]:
account_df = transactions_count.join(transactions_timestamps, "address_id", "outer")
account_df = account_df.join(contracts_created, "address_id", "outer")
account_df = account_df.fillna({"contracts_created": 0})
account_df = account_df.withColumn("total_tx_with_contracts", 
                   F.col("outgoing_tx_count") + F.col("incoming_tx_count") + F.col("contracts_created"))
account_df = account_df.join(unique_received_transactions, "address_id", "left")
account_df = account_df.fillna({"unique_received_transactions": 0})
account_df = account_df.join(eth_transactions, "address_id", "left").fillna(0)
account_df = account_df.join(total_erc_20_tnx, "address_id", "left").fillna(0)
account_df = account_df.join(erc20eth_out, "address_id", "left").fillna(0)
account_df = account_df.join(min_erc20token_in, "address_id", "left").fillna(0)
account_df = account_df.join(num_unique_erc20tokens_out, "address_id", "left").fillna(0)

In [12]:
#count null values in each column
account_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in account_df.columns]).show()

+----------+-----------------+-----------------+--------------------------+-----------------+-----------------------+----------------------------+------------------+----------------+-----------------------+-----------------+--------------------------+
|address_id|outgoing_tx_count|incoming_tx_count|time_difference_in_minutes|contracts_created|total_tx_with_contracts|unique_received_transactions|total_incoming_eth|total_erc_20_tnx|total_outgoing_erc20eth|min_erc20token_in|num_unique_erc20tokens_out|
+----------+-----------------+-----------------+--------------------------+-----------------+-----------------------+----------------------------+------------------+----------------+-----------------------+-----------------+--------------------------+
|         0|                0|                0|                         0|                0|                      0|                           0|                 0|               0|                      0|                0|                    

In [13]:
account_df.write.mode('overwrite').parquet("data/parquet_files/account_df.parquet")

In [14]:
account_data = spark.read.parquet("data/parquet_files/account_df.parquet")
account_data.show()

+----------+-----------------+-----------------+--------------------------+-----------------+-----------------------+----------------------------+-------------------+----------------+-----------------------+-----------------+--------------------------+
|address_id|outgoing_tx_count|incoming_tx_count|time_difference_in_minutes|contracts_created|total_tx_with_contracts|unique_received_transactions| total_incoming_eth|total_erc_20_tnx|total_outgoing_erc20eth|min_erc20token_in|num_unique_erc20tokens_out|
+----------+-----------------+-----------------+--------------------------+-----------------+-----------------------+----------------------------+-------------------+----------------+-----------------------+-----------------+--------------------------+
|         9|               34|               27|         38464.28333333333|                0|                     61|                          22| 1.4637957233935595|               5|                    0.0|        13322.724|                